<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Random Search로 Decision Tree 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/boston.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [3]:
# 상위 몇 개 행 확인
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
# 기초통계량 확인
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [5]:
# target 확인
target = 'medv'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 학습용, 평가용 데이터 분리**

In [6]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 확인
print(x_train.shape)
print(x_test.shape)

(354, 13)
(152, 13)


# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 모델 선언시 max_depth=5, random_state=2022를 지정합니다.

In [7]:
# 불러오기

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [8]:
# 선언하기

model = DecisionTreeRegressor(max_depth=5, random_state=2022)

In [9]:
# 학습하기

model.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=2022)

In [10]:
# 예측하기

y_pred = model.predict(x_test)

In [11]:
# 평가하기

print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 3.2522706568198876
R2-Score: 0.7147610837328198


# 5.성능 튜닝


- Random Search로 튜닝을 진행합니다. 


**1) 모델링 & 튜닝**

In [12]:
# 파라미터 선언
  # max_depth: 1~50

rand_param = {'max_depth':range(1, 51)}

In [13]:
# 불러오기

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [14]:
# 선언하기
model_dst = DecisionTreeRegressor(random_state=2022)

# Random Search 선언
  # cv=10
  # n_iter=30
  # scoring='r2'
model = RandomizedSearchCV(model_dst,
                          rand_param, 
                          cv=10, 
                          n_iter=30,
                          scoring='r2', 
                          n_jobs=-1)

In [15]:
# 학습하기

model.fit(x_train, y_train)

RandomizedSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=2022),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'max_depth': range(1, 51)},
                   scoring='r2')

**2) 결과 확인**

In [16]:
# mean_test_score 확인

model.cv_results_

{'mean_fit_time': array([0.01107044, 0.01216784, 0.01067286, 0.01206596, 0.00748115,
        0.00967333, 0.0094743 , 0.0136631 , 0.00977256, 0.01206658,
        0.01196754, 0.01126895, 0.01186786, 0.01107028, 0.00997286,
        0.00678177, 0.00997283, 0.01087036, 0.01296473, 0.00648274,
        0.01206665, 0.00987363, 0.00997314, 0.00817738, 0.01316438,
        0.01226728, 0.01805091, 0.01037152, 0.01376262, 0.01033728]),
 'std_fit_time': array([0.00137217, 0.00139722, 0.0016108 , 0.0018619 , 0.00102348,
        0.00077899, 0.00102165, 0.00889852, 0.00107421, 0.00346835,
        0.00401421, 0.00240438, 0.00257945, 0.00379608, 0.00089199,
        0.00139605, 0.0020438 , 0.00319968, 0.00388844, 0.00066917,
        0.00242118, 0.00094112, 0.00089223, 0.00171622, 0.00417934,
        0.00726163, 0.02300485, 0.00119573, 0.01046694, 0.00307898]),
 'mean_score_time': array([0.00458686, 0.00518656, 0.00598278, 0.0050864 , 0.00408831,
        0.00388987, 0.00468764, 0.00389061, 0.00418961, 0.00

In [17]:
# 최적 파라미터

print(model.best_params_)

{'max_depth': 6}


In [18]:
# 최적 파라미터 사용시 성능

print(model.best_score_)

0.750586066038991


**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [19]:
# 예측하기

y_pred = model.predict(x_test)

In [20]:
# 평가하기

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

3.289997930052067
0.7155124286118656


# 6.성능 튜닝


- Grid Search로 튜닝을 진행합니다. 


**1) 모델링 & 튜닝**

In [21]:
# 파라미터 선언
  # max_depth: 1~50

grid_param = {'max_depth':range(1,51)}

In [22]:
# 불러오기

from sklearn.model_selection import GridSearchCV

In [23]:
# 선언하기
model_grid = DecisionTreeRegressor(random_state=2022)

# Random Search 선언
  # cv=10
  # n_iter=30
  # scoring='r2'
model = GridSearchCV(model_grid,
                     grid_param, 
                     cv=10,
                     scoring='r2', 
                     n_jobs=-1)

In [24]:
# 학습하기

model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=2022),
             n_jobs=-1, param_grid={'max_depth': range(1, 51)}, scoring='r2')

**2) 결과 확인**

In [25]:
# mean_test_score 확인

model.cv_results_['mean_test_score']

array([0.34989816, 0.64387954, 0.6747913 , 0.70591332, 0.74343071,
       0.75058607, 0.71479741, 0.72103549, 0.71935341, 0.69424064,
       0.684109  , 0.68481433, 0.66897665, 0.71325998, 0.68692905,
       0.68845066, 0.64117459, 0.68980603, 0.64043612, 0.67170607,
       0.66387521, 0.66814223, 0.66904156, 0.67043127, 0.67043127,
       0.67043127, 0.67043127, 0.67043127, 0.67043127, 0.67043127,
       0.67043127, 0.67043127, 0.67043127, 0.67043127, 0.67043127,
       0.67043127, 0.67043127, 0.67043127, 0.67043127, 0.67043127,
       0.67043127, 0.67043127, 0.67043127, 0.67043127, 0.67043127,
       0.67043127, 0.67043127, 0.67043127, 0.67043127, 0.67043127])

In [26]:
# 최적 파라미터

print(model.best_params_)

{'max_depth': 6}


In [27]:
# 최적 파라미터 사용시 성능

print(model.best_score_)

0.750586066038991


**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [28]:
# 예측하기

y_pred = model.predict(x_test)

In [29]:
# 평가하기

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

3.289997930052067
0.7155124286118656
